In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

## Import relevant libraries

In [2]:
!pip install gensim==3.8.1

from gensim.models import Word2Vec

In [3]:
# core system imports
import os

import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import string
import random
import joblib
from unidecode import unidecode
import itertools
from timeit import timeit

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Dense, 
    Input, 
    LSTM, 
    Embedding, 
    Dropout, 
    Activation, 
    Bidirectional, 
    GlobalMaxPool1D
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import (
    initializers, 
    regularizers, 
    constraints, 
    optimizers, 
    layers
)

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
    precision_recall_fscore_support
)

# Matplotlib config
%matplotlib inline
%alias_magic t timeit


Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [34]:
# check for available GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [29]:
# Location of the pipeline metadata store
_pipeline_root = '../pipeline/'

# Directory of the raw data files
_data_root = '../input/'

# Directory of the pretrained word embeddings
_embedding_root = '../input/embeddings'

_data_filepath = os.path.join(_data_root, "data.csv")
_stopwords_filepath = os.path.join(_data_root, "stopwords.txt")

In [30]:
# List input datasets in directory
os.listdir(_data_root)

# Word embeddings
print(os.listdir(_embedding_root))

['model_sg', 'model_sg.trainables.syn1neg.npy', 'model_sg.wv.vectors.npy']


## Load training and test data

In [37]:
# Read data from CSV file
data = pd.read_csv(_data_filepath)

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   text       99 non-null     object 
 1   author_id  99 non-null     float64
 2   Label      99 non-null     object 
dtypes: float64(1), object(2)
memory usage: 2.4+ KB


In [35]:
data.head()

,text,author_id,Label
0,ishmbuhri t nemi bbbn sufeton yn snd d y mgnc...,7.970000e+17,Neutral
1,duk dn bokon d bid ilimin ddini nnob ne cikin ...,2.290470e+09,Neutral
2,duk mutumin d yyi tunnin bw mutne ilimi d koy ...,1.071387e+09,Neutral
3,duk wnd y sbw dokr kucew kmuw dg cutr coron zi...,1.260000e+18,Positive
4,duk wnd y sn y fito dg ynkin d ke nnobr coron ...,1.039268e+09,Positive


In [22]:
# Read stop words
stopwords_list = list()

with open(_stopwords_filepath) as file:
    stopwords_list = [line.strip() for line in file.readlines()]

In [23]:
listToStr = ' '.join([str(word) for word in stopwords_list])
print(listToStr)

ta da ya sai ba yi na kuma ma ji cikin in ni wata wani ce tana don za sun amma ga ina ne mai suka wannan a ko lokacin su take shi yake yana ka ban ita tafi


In [38]:
# Read saved data from disk
def load_pickle(filename):
    data = joblib.load(filename)
    return(data)
    
# Save data to disk for future use
def save_pickle(data, filename):
    joblib.dump(data, filename)
    return True

In [24]:
# removing stopwords
def _apply_lowercase(text):
    text = [item for item in text if item not in stopwords_list]
    text = ''.join(text)
    return text

# removing stopwords
def _stopwords_removal(text):
    text = [item for item in text if item not in stopwords_list]
    text = ''.join(text)
    return text

# remove punctuations
def _punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str

# Shuffle dataset
def _shuffle_dataset(dataset):
    dataset = shuffle(data)
    dataset = data.reset_index(drop=True)
    return dataset

In [25]:
# Change string to lower case
data['text'] = data['text'].apply(str.lower)

# remove punctuations or special characters
data['text'] = data['text'].apply(_punctuation_removal)

# remove stopwords
data['text'] = data['text'].apply(_stopwords_removal)

# Shuffle the dataset to prevent bias:
data = _shuffle_dataset(data)

# Print head of the data
data.head()

,text,author_id,Label
0,ishmbuhri t nemi bbbn sufeton yn snd d y mgnc...,7.970000e+17,Neutral
1,duk dn bokon d bid ilimin ddini nnob ne cikin ...,2.290470e+09,Neutral
2,duk mutumin d yyi tunnin bw mutne ilimi d koy ...,1.071387e+09,Neutral
3,duk wnd y sbw dokr kucew kmuw dg cutr coron zi...,1.260000e+18,Positive
4,duk wnd y sn y fito dg ynkin d ke nnobr coron ...,1.039268e+09,Positive


### Split Dataset

+ Tranining: 70% of the dataset
+ Testing: 30% of dataset

In [40]:
list_classes = ["Label"]

X_train, X_test, Y_train, Y_test = train_test_split(data["text"], data[list_classes], test_size=0.3, random_state = 1)

Y_train = Y_train.values
Y_test = Y_test.values

# Show dimension of the comments
X_train.shape, X_test.shape

((69,), (30,))

### Tokenize Sentences

To be able to train our model with a text data, we'd have to convert it into number form, for this we're going to use the Tokenizer module from Keras.preprocessing library

In [47]:
list_sentences_train = X_train.values
list_sentences_test = X_test.values

In [48]:
tokenizer = Tokenizer(lower=True)

tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [49]:
# size of training vocabulary (number of unique words)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

754

In [50]:
# print out a random sequence of text from the tokenized training data
print(random.choice(list_tokenized_train))

[181, 61, 1, 14, 1, 9, 1, 15, 3, 4, 182, 90]


In [51]:
print(len(list_tokenized_train), 'train sequences')
print(len(list_tokenized_test), 'test sequences')

print('Average train sequence length: {}'.format(np.mean(list(map(len, list_tokenized_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, list_tokenized_test)), dtype=int)))

69 train sequences
30 test sequences
Average train sequence length: 20
Average test sequence length: 10


## Pad tokenized Sentences

You might have observed that the sentences are not of the same lengths, so we need to pad them with zeros (0's) so that the resulting array will have equal length.

We'd use `text` module from `keras.preprocessing` library

We'd use a max character of 300

In [52]:
maxlen = 300
X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [53]:
print(X_train[:1])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 